# Network of Things demo

Start a Mosquitto container first. For example:
- Use `codes\_demo\1_start_broker.sh` to start a Mosquitto container on Raspberry Pi.
- Config files are in `mqtt_config\mqtt`.
- set `allow_anonymous true` in `mqtt_config\mqtt\config\mosquitto.conf` to allow anonymous client.

## Getting Started
What this notebook does:  
- Using a client on PC
- List connected nodes
- Send messages to remote nodes:
 - Return results (read GPIOs)via RPC mechanism.
 - Write data to remote nodes (write GPIOs).
 - Execute arbitrary code on remote nodes.

In [1]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'micropython')))
 
import client
from collections import OrderedDict

### Start client

In [2]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

My name is Client_366

Sending 277 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2018-04-06 16:59:11.653450'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2018-04-06 16:59:11.653450'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


[Connected: ('123.240.210.68', 1883)]
[Listen to messages]
Node not ready yet.


### List of nodes

In [3]:
remote_nodes = ['n_Alpha', 'n_Lambda', 'n_Beta']
remote_nodes

['n_Alpha', 'n_Lambda', 'n_Beta']

In [4]:
targeted_node = 'n_Lambda'
targeted_node

'n_Lambda'

### Prepare messages

In [4]:
messages = OrderedDict()

___
## DEMOs
___

### Write GPIOs

In [6]:
messages['write_GPIOs'] = {'message_type': 'command',
                           'command': 'write GPIOs',
                           'kwargs': {'pins_and_values': [(2, 0),]}} 

the_client.request(targeted_node, messages['write_GPIOs']);


Sending 260 bytes
Message:
OrderedDict([('command', 'write GPIOs'), ('correlation_id', '2017-09-27 22:31:37.768600'), ('kwargs', {'pins_and_values': [(2, 0)]}), ('message_id', '2017-09-27 22:31:37.768600'), ('message_type', 'command'), ('receiver', 'n_Lambda'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



In [7]:
messages['write_GPIOs'] = {'message_type': 'command',
                           'command': 'write GPIOs',
                           'kwargs': {'pins_and_values': [(2, 1),]}} 

the_client.request(targeted_node, messages['write_GPIOs']);


Sending 260 bytes
Message:
OrderedDict([('command', 'write GPIOs'), ('correlation_id', '2017-09-27 22:31:39.573200'), ('kwargs', {'pins_and_values': [(2, 1)]}), ('message_id', '2017-09-27 22:31:39.573200'), ('message_type', 'command'), ('receiver', 'n_Lambda'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



### Blink each node
[YouTube video clip](https://youtu.be/I_s-no_0L24)

In [5]:
messages['blink_led'] = {'message_type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 3, 'on_seconds': 0.1, 'off_seconds': 0.1}}

for remote_node in remote_nodes:
    the_client.request(remote_node, messages['blink_led']);

NameError: name 'remote_nodes' is not defined

### Blink each node (broadcasting)

In [19]:
the_client.request('Hub', messages['blink_led']);


Sending 275 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-09-27 22:32:58.446800'), ('kwargs', {'on_seconds': 0.1, 'times': 3, 'off_seconds': 0.1}), ('message_id', '2017-09-27 22:32:58.446800'), ('message_type', 'command'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 275 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-09-27 22:32:58.446800'), ('kwargs', {'on_seconds': 0.1, 'times': 3, 'off_seconds': 0.1}), ('message_id', '2017-09-27 22:32:58.446800'), ('message_type', 'command'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



### Read GPIOs

In [ ]:
messages['read_GPIOs'] = {'message_type': 'command',
                          'command': 'read GPIOs',
                          'kwargs': {'pins': [5, 12, 13, 14, 15, 16]},
                          'need_result': True}

_, result = the_client.request(targeted_node, messages['read_GPIOs'])
print('\nGPIO status for {}: {}\n'.format(targeted_node, result.get()));

In [ ]:
rpc = the_client.request(targeted_node, messages['read_GPIOs'])[1].get

rpc()

### Read GPIOs from each node

In [ ]:
# for remote_node in remote_nodes:
#     _, result = the_client.request(remote_node, messages['read_GPIOs']) 
#     print('\nGPIO status for {}: {}\n'.format(remote_node, result.get()))

### Read GPIOs from each node and queue async_results

In [ ]:
status = []

for remote_node in remote_nodes:
    _, result = the_client.request(remote_node, messages['read_GPIOs']) 
    status.append((remote_node, result))
    
status

### Get results via async_results

In [ ]:
for remote_node, result in status: 
    print('\nGPIO status for {}: {}\n'.format(remote_node, result.get()))

### Eval

In [ ]:
messages['test_eval'] = {'message_type': 'eval',
                         'to_evaluate': '2+3',
                         'need_result': True}

_, result = the_client.request(targeted_node, messages['test_eval']) 
print('result:', result.get());

### Exec

In [ ]:
messages['test_exec'] = {'message_type': 'exec',
                         'to_exec': 'print("Hello World!")'} 

the_client.request(targeted_node, messages['test_exec']);

### Invoke remote functions

In [ ]:
# messages['test_function'] = {'message_type': 'function',
#                              'function': 'blink_led',
#                              'kwargs': {'times': 3, 'on_seconds': 0.1, 'off_seconds': 0.1}}

# the_client.request(targeted_node, messages['test_function']);

### Transmit function (behavior) and invoke it remotely

In [ ]:
with open('functions_def.py') as f:
    script = f.read()    
the_client.request(targeted_node, {'message_type': 'script', 
                                   'script': script}) 

the_client.request(targeted_node, {'message_type': 'exec',
                                   'to_exec': "import script"})

the_client.request(targeted_node, {'message_type': 'exec',
                                   'to_exec': "script.function_to_test('_______ Testing dynamic function ______')"});

### Upload and run a script file

In [ ]:
# with open('script_to_deploy.py') as f:
#     script = f.read()
    
# messages['test_upload_script'] = {'message_type': 'script', 
#                                   'script': script} 

# the_client.request(targeted_node, messages['test_upload_script']);

### Stop the demo

In [3]:
# Stopping
the_client.stop()
the_client = None
print('\n[________________ Demo stopped ________________]\n')

[Closed: ('123.240.210.68', 1883)]

[________________ Demo stopped ________________]

